# Querying wikidata

In [1]:
from pprint import pprint
from SPARQLWrapper import SPARQLWrapper, JSON, RDF

In [2]:
sparql = SPARQLWrapper("http://query.wikidata.org/sparql")
sparql.setReturnFormat(JSON)

## Querying company names

In [3]:
"""
SELECT DISTINCT ?id ?idLabel WHERE {
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en". }
    VALUES ?orgLike { 
        wd:Q93429702 
        wd:Q22687 
        wd:Q16691582 
        wd:Q4830453 
        wd:Q891723 
        wd:Q6881511
    }
    
    ?id wdt:P31 ?orgLike .
  
} LIMIT 20
"""

'\nSELECT DISTINCT ?id ?idLabel WHERE {\n    SERVICE wikibase:label { bd:serviceParam wikibase:language "ru,en". }\n    VALUES ?orgLike { \n        wd:Q93429702 \n        wd:Q22687 \n        wd:Q16691582 \n        wd:Q4830453 \n        wd:Q891723 \n        wd:Q6881511\n    }\n    \n    ?id wdt:P31 ?orgLike .\n  \n} LIMIT 20\n'

## Querying tickers and names

In [5]:
# Business listed on NYSE and NASDAQ along with their ticker symbols
sparql.setQuery( """
SELECT DISTINCT ?id ?idLabel ?exchangesLabel ?ticker WHERE {
    SERVICE wikibase:label { bd:serviceParam wikibase:language "en" . }
    VALUES ?exchanges { wd:Q13677 wd:Q82059 }
    
    ?id p:P414 ?exchange .
        
    ?exchange ps:P414 ?exchanges ;
              pq:P249 ?ticker .
} LIMIT 20 """)

results = sparql.query().convert()

In [6]:
pprint(results["results"]["bindings"][0])

{'exchangesLabel': {'type': 'literal',
                    'value': 'New York Stock Exchange',
                    'xml:lang': 'en'},
 'id': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q522617'},
 'idLabel': {'type': 'literal', 'value': 'Invesco', 'xml:lang': 'en'},
 'ticker': {'type': 'literal', 'value': 'IVZ'}}


In [40]:
print("[Q]             [COMPANY]                             [TICKER]  [AT EXCHANGE]")
for company in results["results"]["bindings"]:
    company_entity = company["id"]["value"].split("/")[-1].ljust(15)
    company_name = company["idLabel"]["value"].ljust(35)
    exchange = company["exchangesLabel"]["value"]
    ticker = company["ticker"]["value"]
    print(company_entity, company_name, ticker, exchange, sep="\t")

[Q]             [COMPANY]                             [TICKER]  [AT EXCHANGE]
Q7271499       	Questar Corporation                	STR	Нью-Йоркская фондовая биржа
Q5206232       	DTE Energy                         	DTE	Нью-Йоркская фондовая биржа
Q5207184       	DaVita                             	DVA	Нью-Йоркская фондовая биржа
Q7688650       	Taubman Centers                    	TCO	Нью-Йоркская фондовая биржа
Q18344949      	Globus Medical                     	GMED	Нью-Йоркская фондовая биржа
Q5214701       	Dana Corporation                   	DAN	Нью-Йоркская фондовая биржа
Q5337641       	Edgewell Personal Care             	EPC	Нью-Йоркская фондовая биржа
Q5599779       	Great Plains Energy                	GXP	Нью-Йоркская фондовая биржа
Q4833115       	Azurix                             	AZX	Нью-Йоркская фондовая биржа
Q7567571       	South Jersey Industries            	SJI	Нью-Йоркская фондовая биржа
Q7440621       	Sealed Air                         	SEE	Нью-Йоркская фондовая бир

## Querying company by name

In [18]:
# Entities that are listen on stockexchange and match by name
query =  """
SELECT DISTINCT ?id ?label WHERE {
    VALUES ?exchanges { wd:Q13677 wd:Q82059 }
    
    ?id p:P414 ?exchange;
        rdfs:label ?label FILTER (regex(?label, "%s", "i") && lang(?label)="ru") .
    
} LIMIT 20 """ % ("apple",)
sparql.setQuery(query)

results = sparql.query().convert()

In [19]:
pprint(results["results"]["bindings"])

[{'id': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q312'},
  'label': {'type': 'literal', 'value': 'Apple', 'xml:lang': 'ru'}},
 {'id': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q1254149'},
  'label': {'type': 'literal',
            'value': 'Dr Pepper Snapple Group',
            'xml:lang': 'ru'}}]


## Querying company by ticker

Some useful ids
+ `ticker symbol (P249)`
+ `ISIN (P946)`
+ `company (Q783794)`
    + legal entity representing an association of people, whether natural, legal or a mixture of both, with a specific objective
+ `organization (Q43229)`
    + social entity (not necessarily commercial) uniting people into a structured group managing shared means to meet some needs, or to pursue collective goals

In [3]:
# Entities that are listen on stockexchange and match by ticker
query =  """
SELECT DISTINCT ?id ?idLabel ?alias WHERE {
    SERVICE wikibase:label { bd:serviceParam wikibase:language "ru,en". }
    VALUES ?ticker { "%s"}
    
    ?id p:P414 ?exchange.
    ?exchange pq:P249 ?ticker.
    ?id skos:altLabel ?alias .
}
LIMIT 150""" % ("SBER",)
sparql.setQuery(query)

results = sparql.query().convert()

In [4]:
pprint(results["results"]["bindings"])

[{'alias': {'type': 'literal', 'value': 'Sberbank', 'xml:lang': 'en'},
  'id': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q205012'},
  'idLabel': {'type': 'literal', 'value': 'Сбербанк РФ', 'xml:lang': 'ru'}},
 {'alias': {'type': 'literal',
            'value': 'Акционерный коммерческий Сберегательный банк Российской '
                     'Федерации',
            'xml:lang': 'ru'},
  'id': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q205012'},
  'idLabel': {'type': 'literal', 'value': 'Сбербанк РФ', 'xml:lang': 'ru'}},
 {'alias': {'type': 'literal',
            'value': 'ОАО «Сбербанк России»',
            'xml:lang': 'ru'},
  'id': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q205012'},
  'idLabel': {'type': 'literal', 'value': 'Сбербанк РФ', 'xml:lang': 'ru'}},
 {'alias': {'type': 'literal', 'value': 'ПАО Сбербанк', 'xml:lang': 'ru'},
  'id': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q205012'},
  'idLabel': {'type': 'literal', 'v

### Query label by QID

In [17]:
query =  """
SELECT DISTINCT ?label WHERE {
    wd:%s rdfs:label ?label FILTER (lang(?label)="ru" || lang(?label)="en").
}
LIMIT 20""" % ("Q380")
sparql.setQuery(query)

results = sparql.query().convert()

In [18]:
pprint(results["results"]["bindings"])

[{'label': {'type': 'literal', 'value': 'Facebook, Inc.', 'xml:lang': 'ru'}},
 {'label': {'type': 'literal', 'value': 'Facebook, Inc.', 'xml:lang': 'en'}}]


In [22]:
results

{'head': {'vars': ['label']},
 'results': {'bindings': [{'label': {'xml:lang': 'ru',
     'type': 'literal',
     'value': 'Русал'}}]}}

In [25]:
pprint(results["results"]["bindings"][0]["label"]["value"])

'Русал'
